
# 🕹️ Emoji → Micro-Game (Colab, fixed)

Give a few emojis and it will generate a tiny HTML5 canvas game you can play inline: arrows to move, space to shoot.


## 🎮 Generate game

In [ ]:

from IPython.display import IFrame, display
from pathlib import Path

TITLE = "Rocket vs Zombies"
EMOJIS = "🚀🧟‍♂️🎯"
player_emoji = EMOJIS[0] if EMOJIS else "🚀"
enemy_emoji = "🧟‍♂️"
bullet_emoji = "🔴"

html = f'''
<!DOCTYPE html>
<html>
<meta charset="UTF-8"/>
<title>{TITLE}</title>
<style>
  html, body {{ margin:0; background:#0b0b15; color:white; font-family:system-ui; }}
  #c {{ display:block; margin:0 auto; background:#101827; border:2px solid #334155; }}
  #hud {{ text-align:center; padding:10px; }}
</style>
<canvas id="c" width="640" height="400"></canvas>
<div id="hud"><b>{TITLE}</b> — Arrows to move, Space to shoot</div>
<script>
const cv = document.getElementById('c');
const cx = cv.getContext('2d');
let keys = {{}};
document.addEventListener('keydown', e=>keys[e.code]=true);
document.addEventListener('keyup', e=>keys[e.code]=false);

const player = {{ x: 320, y: 340, vx:0, vy:0, lives:3, score:0 }};
let bullets = [];
let enemies = [];
let t = 0;

function spawnEnemy() {{
  enemies.push({{ x: Math.random()*620+10, y: -20, vy: 0.8+Math.random()*0.6 }});
}}

function shoot() {{
  bullets.push({{ x: player.x, y: player.y-10, vy: -4 }});
}}

let fireLatch = false;
function step() {{
  t++;
  const speed = 2.3;
  if (keys['ArrowLeft']) player.x -= speed;
  if (keys['ArrowRight']) player.x += speed;
  if (keys['ArrowUp']) player.y -= speed;
  if (keys['ArrowDown']) player.y += speed;
  player.x = Math.max(16, Math.min(624, player.x));
  player.y = Math.max(20, Math.min(380, player.y));

  if (keys['Space']) {{ if (!fireLatch) {{ shoot(); fireLatch = true; }} }} else fireLatch = false;

  if (t % 45 === 0) spawnEnemy();
  enemies.forEach(e => e.y += e.vy); enemies = enemies.filter(e => e.y < 430);
  bullets.forEach(b => b.y += b.vy); bullets = bullets.filter(b => b.y > -10);

  for (let i=enemies.length-1; i>=0; --i) {{
    const e = enemies[i];
    if (Math.hypot(e.x - player.x, e.y - player.y) < 18) {{ player.lives--; enemies.splice(i,1); if (player.lives <= 0) gameOver(); }}
    for (let j=bullets.length-1; j>=0; --j) {{
      const b = bullets[j];
      if (Math.hypot(e.x - b.x, e.y - b.y) < 16) {{ bullets.splice(j,1); enemies.splice(i,1); player.score += 10; break; }}
    }}
  }}

  cx.fillStyle = '#0b1220'; cx.fillRect(0,0,640,400);
  cx.font = '24px serif'; cx.fillText('{player_emoji}', player.x-12, player.y+8);
  enemies.forEach(e => cx.fillText('{enemy_emoji}', e.x-12, e.y+8));
  bullets.forEach(b => cx.fillText('{bullet_emoji}', b.x-8, b.y+8));
  cx.fillStyle = '#fff'; cx.font = '14px monospace';
  cx.fillText('Score: ' + player.score + '   Lives: ' + player.lives, 10, 18);
  requestAnimationFrame(step);
}}

function gameOver() {{
  alert('Game Over! Score: ' + player.score);
  enemies = []; bullets = []; player.x=320; player.y=340; player.lives=3; player.score=0;
}}

step();
</script>
</html>
''';

Path("results").mkdir(exist_ok=True)
Path("results/emoji_game.html").write_text(html, encoding="utf-8")
display(IFrame(src="results/emoji_game.html", width=660, height=480))
